In [1]:
import threading
import time
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODA
#include "gpio.h"
#include "pyprintf.h"
#include <unistd.h>
static int inited = 0;
static gpio pins[8];
void init_pmoda()
{
    if(inited)
    return;
    for(int i = 0; i < 8; i++)
    {
        pins[i] = gpio_open(i);
        gpio_set_direction(pins[i], GPIO_OUT);
        gpio_write(pins[i], 0);
    }
    inited = 1;
}
//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val)
{
    if (val > 1)
    {
        pyprintf("pin value must be 0 or 1");
    }
    if(!inited)
    {
        init_pmoda();
    }
    gpio_write(pins[pin], val);
}

// reset GPIO bins meaning writing 0 as output to all pins 0 - 7
void reset_pin(unsigned int pin)
{
    if(!inited)
    {
        init_pmoda();
    }
    write_gpio(pin, 0);
}

void control_led4(unsigned int total_us, unsigned int frequency)
{
    if(!inited)
    {
        init_pmoda();
    }
    unsigned int period_us = 1000000u / frequency;
    unsigned int half_us = period_us / 2u;
    unsigned int cycles = total_us / period_us;
    for(unsigned int i = 0; i < cycles; i++)
    {
        write_gpio(2, 1);
        usleep((useconds_t)half_us);
        write_gpio(2, 0);
        usleep((useconds_t)half_us);
    }
}

# Code for blinking LED
4 light from the board and one from LED, writing function to blink them

In [3]:
def blink(led_idx, total_s, frequency):
    if total_s <= 0:
        base.leds[led_idx].off()
        return
    if frequency <= 0:
        raise ValueError("frequency must greater than 0")
    if led_idx < 0 or led_idx > 4:
        raise ValueError("wrong led input")
    period = 1 / frequency
    half_period = 0.5 * period

    cycles = int(total_s * frequency)
    if led_idx <= 3:
        for _ in range(cycles):
            base.leds[led_idx].on()
            time.sleep(half_period)
            base.leds[led_idx].off()
            time.sleep(half_period)
    else:
        control_led4(int(total_s * 1000000), int(frequency))
        time.sleep(total_s) # adding to debug !

def led_off(led_idx):
    if led_idx <= 3:
        base.leds[led_idx].off()
    else:
        write_gpio(2, 0)

In [4]:
blink(4, 3, 10)
#led_off(4)
#blink(3, 1.5, 4)

# Five Thread Five Lock
core: 1. to avoid deadlock one of the person need to have opposite order of getting folks. For
example 4 people first try lock on their left, while 1 people first try lock on their right
core: 2. if a person get the first lock successfully but get second lock unsuccessfully, he needs to
give up both lock to prevent starvation core: 3. to make people not starve too much, napping
time should be longer than starvation time, so that other people could get more chance of
getting a lock

In [5]:
import threading, time, random

# define print msg, using lock to protect as well
print_lock = threading.Lock()
def log(msg):
    ts = time.time()
    with print_lock:
        print(f"[{ts:.3f}] {msg}") #learning from online resources, Kevin Noted in lab
                                    #acquire print lock in blocking way, and later rele
'''
person 0 --> Lock 0 , Lock 4
person 1 --> Lock 1 , Lock 0
person 2 --> Lock 2 , Lock 1
person 3 --> Lock 3 , Lock 2
person 4 --> Lock 3 , Lock 4
'''
def people(_l0, _l1, _l2, _l3, _l4, person_id):
    lock_id = {_l0: 0, _l1: 1, _l2: 2, _l3: 3, _l4: 4} # For print mapping
    using_first_lock = False
    using_second_lock = False
    if person_id == 0:
        first_lock = _l0 # right
        second_lock = _l4 # left
    elif person_id == 1:
        first_lock = _l1 # right 
        second_lock = _l0 # left
    elif person_id == 2:
        first_lock = _l2 # right
        second_lock = _l1 # left
    elif person_id == 3: 
        first_lock = _l3 # right
        second_lock = _l2 # left
    elif person_id == 4:
        first_lock = _l3 # left
        second_lock = _l4 # right
    while not stop_event.is_set():
        # starving mode, LED off until potentially get lock
        led_off(person_id);
        using_first_lock = first_lock.acquire(True)
        using_second_lock = second_lock.acquire(False)
        if(using_second_lock): #non blocking for second lock
            #going into eating mode
            try:
                log(f"philosopher {person_id} got first lock {lock_id[first_lock]} and") 
                #blink(person_id, 2, 48)
                blink(person_id, random.uniform(1.8, 2.2), 48)
            finally:
                first_lock.release()
                second_lock.release()
                #go into napping mode
                #blink(person_id, 2.5, 12)
                blink(person_id,random.uniform(2.4,2.6),12)
        else:
            #give up first lock
            first_lock.release()
            #going into starving mode, but add a small randomly delay
            time.sleep(random.uniform(0.001,0.005))
                
def button_terminate():
    while not stop_event.is_set():
        if base.buttons[0].read() == 1:
            stop_event.set()
            break
    time.sleep(0.1)

stop_event = threading.Event()

# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()

for i in range(5):
    t = threading.Thread(target=people, args=(fork0, fork1, fork2, fork3, fork4, i), name = f"philosopher-{i}")
    threads.append(t)
    t.start()

terminate = threading.Thread(target = button_terminate , name = "terminate")
terminate.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))
for i in range(4):
    base.leds[i].off()
write_gpio(2,0)

[1746748149.484] philosopher 0 got first lock 0 and
[1746748149.491] philosopher 2 got first lock 2 and


/tmp/ipykernel_1922/115043116.py:85: DeprecationWarning: getName() is deprecated, get the name attribute instead
  name = t.getName()


[1746748152.702] philosopher 3 got first lock 3 and
[1746748152.765] philosopher 1 got first lock 1 and
[1746748155.637] philosopher 0 got first lock 0 and
[1746748155.919] philosopher 2 got first lock 2 and
[1746748158.855] philosopher 4 got first lock 3 and
[1746748159.347] philosopher 1 got first lock 1 and
[1746748160.852] philosopher 3 got first lock 3 and
[1746748162.123] philosopher 0 got first lock 0 and
[1746748163.756] philosopher 2 got first lock 2 and
[1746748165.292] philosopher 4 got first lock 3 and
[1746748166.705] philosopher 1 got first lock 1 and
[1746748167.404] philosopher 3 got first lock 3 and
[1746748170.057] philosopher 0 got first lock 0 and
[1746748170.432] philosopher 2 got first lock 2 and
[1746748173.431] philosopher 3 got first lock 3 and
[1746748173.458] philosopher 1 got first lock 1 and
[1746748176.328] philosopher 4 got first lock 3 and
[1746748176.359] philosopher 2 got first lock 2 and
[1746748178.539] philosopher 0 got first lock 0 and
[1746748179.